In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls
!rm -fr darknet
!git clone https://github.com/AlexeyAB/darknet/
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!apt update
!apt install cuda-8-0 -y
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
!apt install gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++
!apt-get install libopencv-dev

gdrive	sample_data
Cloning into 'darknet'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 8482 (delta 10), reused 14 (delta 6), pack-reused 8450
Receiving objects: 100% (8482/8482), 8.71 MiB | 20.36 MiB/s, done.
Resolving deltas: 100% (5709/5709), done.
--2019-01-28 03:15:08--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?8M9lQxc4npE5pKgDpT3YokgPXntGuJYZm62sozKc1jK14fVndA536Mt-mxdWRckbYmshSIGL778ajEcvgqc3PrEnbcP4cuWIPTh8E6YI3FS8WyKviti_tpG9jm1J5OCZn1

In [3]:
%cd darknet

/content/darknet


In [0]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

In [5]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -Iinclude/  -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/http_stream.cpp -o obj/http_stream.o
In file included from ./src/http_stream.h:4:0,
                 from ./src/http_stream.cpp:1:
include/darknet.h:48:16: warning: ‘typedef’ was ignored in this declaration
 typedef struct network_state;
                ^
./src/http_stream.cpp: In member function ‘bool MJPG_sender::write(const cv::Mat&)’:
./src/http_stream.cpp:255:23: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]
                 if (n < outlen)
                       ^
./src/http_stream.cpp: In member function ‘void JSON_sender::close_all()’:
./src/http_stream.cpp:324:20: warning: ISO C++ forbids converting a string constant to ‘char*’ [-Wwrite-strings

In [6]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2019-01-28 03:18:42--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  64.9MB/s    in 3.8s    

2019-01-28 03:18:46 (62.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [7]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg -dont_show

layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64 1.595 BF
   2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32 0.177 BF
   3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128 1.595 BF
   6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
  10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256 1.595 BF
  13 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128 0.177 BF
  14 c

In [8]:
!wget https://pjreddie.com/media/files/darknet19_448.conv.23

--2019-01-28 03:19:01--  https://pjreddie.com/media/files/darknet19_448.conv.23
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79327120 (76M) [application/octet-stream]
Saving to: ‘darknet19_448.conv.23’

darknet19_448.conv. 100%[===================>]  75.65M  54.9MB/s    in 1.4s    

2019-01-28 03:19:03 (54.9 MB/s) - ‘darknet19_448.conv.23’ saved [79327120/79327120]



In [0]:
%mv 190128.cfg /content/darknet
%mv obj.data /content/darknet
%mv obj.names /content/darknet
%mv test.txt /content/darknet
%mv train.txt /content/darknet

In [15]:
%cd /content/gdrive/My Drive/Colab Notebooks/190128
%cp images_190128.zip /content/darknet

/content/gdrive/My Drive/Colab Notebooks/190128


In [22]:
!apt-get install unzip
%cd content/darknet
!unzip images_190128.zip -d /content/darknet/images/

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 72 not upgraded.
[Errno 2] No such file or directory: 'content/darknet'
/content/gdrive/My Drive/Colab Notebooks/190128
Archive:  images_190128.zip
  inflating: /content/darknet/images/0001.jpg  
  inflating: /content/darknet/images/0001.txt  
  inflating: /content/darknet/images/0002.jpg  
  inflating: /content/darknet/images/0002.txt  
  inflating: /content/darknet/images/0003.jpg  
  inflating: /content/darknet/images/0003.txt  
  inflating: /content/darknet/images/0004.jpg  
  inflating: /content/darknet/images/0004.txt  
  inflating: /content/darknet/images/0005.jpg  
  inflating: /content/darknet/images/0005.txt  
  inflating: /content/darknet/images/0006.jpg  
  inflating: /content/darknet/images/0006.txt  
  inflating: /content/darknet/images/0007.jpg  
  inflating: /content/darknet/im

In [0]:
%cd /content/darknet
!./darknet detector train obj.data 190125.cfg darknet19_448.conv.23 -dont_show -map